# 연구 및 기사 작성을 위한 에이전트 생성하기

멀티 에이전트 시스템(multi-agent systems)의 기본 개념을 소개하고, CrewAI 프레임워크에 대한 개요를 제공합니다.

In [1]:
# pip install crewai crewai-tools
# %pip install 'crewai[tools]'

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [4]:
from crewai import Agent, Task, Crew

In [5]:
import os
from crew_ai_utils import pretty_print_result, get_openai_api_key

import dotenv

dotenv.load_dotenv()
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")


## 에이전트 생성
- 에이전트를 정의하고, `role` (역할), `goal` (목표) 및 `backstory` (배경 이야기) 정의
- LLM(Large Language Model)은 역할극을 할 때 성능이 더 좋은 것으로 알려져 있음

### Agent: Planner(콘텐츠 기획자)


In [7]:
planner = Agent(
    role="콘텐츠 기획자 (Content Planner)",
    goal="매력적이고 사실에 근거한 콘텐츠를 {topic}(주제)에 대해 기획합니다.",
    backstory="당신은 {topic}(주제)에 대한 블로그 기사 기획 작업을 하고 있습니다."
              "당신은 독자들에게 유용한 정보를 수집하여 독자들이 학습하고 "
              "정보에 입각한 결정을 내릴 수 있도록 돕습니다. "
              "당신의 작업은 콘텐츠 작가(Content Writer)가 이 주제에 대한 기사를 작성하는 데 기초가 됩니다.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer(콘텐츠 작가)

In [8]:
writer = Agent(
    role="콘텐츠 작가 (Content Writer)",
    goal="주제({topic})에 대해 통찰력 있고 사실에 근거한 의견(opinion)을 작성합니다.",
    backstory="당신은 주제({topic})에 대한 새로운 의견(opinion)을 작성하고 있습니다. "
              "당신은 콘텐츠 기획자(Content Planner)의 작업, 즉 개요(outline)와 주제에 대한 관련 맥락(context)을 기반으로 글을 씁니다. "
              "당신은 콘텐츠 기획자(Content Planner)가 제공한 개요(outline)의 주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰력(insights)을 제공하고 콘텐츠 기획자(Content Planner)가 제공한 정보를 통해 이를 뒷받침합니다. "
              "당신은 당신의 진술이 객관적인 진술이 아닌 의견(opinions)일 때 의견(opinion)에서 인정합니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor(편집자)

In [9]:
editor = Agent(
    role="편집자 (Editor)",
    goal="제공된 블로그 게시물을 편집하여 조직의 글쓰기 스타일에 맞춥니다.",
    backstory="당신은 콘텐츠 작가(Content Writer)로부터 블로그 게시물을 받는 편집자입니다. "
              "당신의 목표는 블로그 게시물을 검토하여 저널리즘의 모범 사례를 따르고,"
              "의견이나 주장을 제시할 때 균형 잡힌 관점을 제공하고,"
              "가능한 경우 주요 논쟁 거리가 되는 주제나 의견을 피하는 것입니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Translator(번역자)

In [10]:
translator = Agent(
    role="번역자 (Translator)",
    goal="편집이 완료된 블로그 게시물을 요청한 언어로 정확하게 번역하여 사용자에게 제공합니다.",
    backstory="당신은 편집자(Editor)로부터 블로그 게시물을 받는 번역자입니다. "
              "당신은 다양한 언어에 능통한 숙련된 번역가이며, 특히 IT 도메인에 전문성이 있습니다. "
              "당신의 목표는 콘텐츠의 의미, 어조, 스타일을 유지하면서 텍스트를 정확하게 번역하는 것입니다. "
              "당신은 다양한 분야의 텍스트를 번역한 경험이 있으며, 각 언어의 문화적 뉘앙스를 이해하고 있습니다.",
    allow_delegation=False,
    verbose=True
)

## 작업 생성

- `task`, `description`, `expected_output`, `agent` 정의

### Task: Plan(콘텐츠 계획)

In [11]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.\n"
        "2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.\n"
        "3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.\n"
        "4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.\n"
    ),
    expected_output="개요(outline), 대상 독자 분석(audience analysis), "
        "SEO 키워드(SEO keywords) 및 리소스(resources)를 포함하는 포괄적인 콘텐츠 계획 문서(comprehensive content plan document)입니다.",
    agent=planner,
)

### Task: Write(콘텐츠 작성)

In [12]:
write = Task(
    description=(
        "1. 콘텐츠 계획(content plan)을 활용하여 {topic}에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.\n"
        "2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.\n"
        "3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.\n"
        "4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.\n"
        "5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).\n"
    ),
    expected_output="잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, 각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다.",
    agent=writer,
)

### Task: Edit(콘텐츠 편집)

In [13]:
edit = Task(
    description=(
        "1. 주어진 블로그 게시물(blog post)에서 문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인합니다.\n"
    ),
    expected_output=(
        "잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, "
        "각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다."
    ),
    agent=editor,
)

### Task: Translate(번역)

In [14]:
translate = Task(
    description=(
        "문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인이 끝나면 일본어로 번역합니다.\n"
    ),
    expected_output=(
        "일본어로 번역된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공합니다."
    ),
    agent=translator,
)

## Crew 구성

- 에이전트(Agents)로 Crew를 구성
- 해당 에이전트(agents)가 수행할 작업(tasks)을 전달
    - 이 간단한 예제에서는* 작업(tasks)이 순차적으로 수행 됨.(즉, 서로 종속적)  
    - 따라서 목록에서 작업(task)의 _순서(order)_ 가 중요함.
- `verbose=True`를 설정하면 실행에 대한 모든 로그를 볼 수 있음

In [16]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

crew = Crew(
    agents=[planner, writer, editor, translator],
    tasks=[plan, write, edit, translate],
    process=Process.sequential,
    verbose=True
)

Overriding of current TracerProvider is not allowed


## Running the Crew

In [17]:
result = crew.kickoff(inputs={"topic": "인공지능"})

# Agent: 콘텐츠 기획자 (Content Planner)
## Task: 1. 인공지능에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.
2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.
3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.
4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.



# Agent: 콘텐츠 기획자 (Content Planner)
## Final Answer: 
**Comprehensive Content Plan Document: Artificial Intelligence**

---

**Content Outline**

1. **Introduction**
   - Brief overview of the importance and impact of AI in various sectors.
   - Statement of AI as an evolving technology with significant implications.

2. **Latest Trends in Artificial Intelligence**
   - AI in healthcare: Discuss advancements in AI diagnostics and personalized medicine.
   - AI in autonomous vehicles: Latest improvements in self-driving technology.
   - AI in creative industries: AI-generated content and its implications.

3. **K

- Display the results of your execution as markdown in the notebook.

In [18]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# 人工知能の止められない進化

## はじめに
近年、人工知能（AI）は多様な分野に影響を与える変革の力として浮上しています。医療から自動運転車、そしてクリエイティブ産業に至るまで、AIの統合は効率と革新を再定義する注目に値する進歩を記しています。発展を続ける技術として、AIは様々な分野において重要な影響を及ぼすだけでなく、その可能性と課題に関する必須の議論を促します。

## 人工知能の最新トレンド

### 医療におけるAI
人工知能は診断精度を向上させ、パーソナライズ医療を促進することで、医療を革命的に変えています。AIアルゴリズムは医療画像データをより効率的に分析し、迅速かつ正確な診断を可能にしています。さらに、個々の遺伝プロファイルに基づいた治療計画のカスタマイズは、未来の医療パラダイムにおけるAIの重要な役割を強調し、患者の成果を最適化するテイラードケアを確保します。

### 自動運転車におけるAI
自動運転車の開発は、AI技術の洗練度に密接に結びついています。最近の自動運転技術の改良は、より安全で効率的な輸送システムの約束を加速させました。これらの進歩は革新的な都市モビリティソリューションへの道を切り拓くだけでなく、交通事故の減少や排出物の削減をもたらし、都市景観を再構築します。

### クリエイティブ産業におけるAI
クリエイティブセクターにおけるAIのコンテンツ生成能力は、新たな可能性と議論を生み出しています。AI生成の音楽やアートから自動化されたコンテンツ作成に至るまで、この革新は創造性と著作権の伝統的な概念に挑戦しています。新しい創造的方法を導入する一方で、知的財産権や芸術的誠実性について重要な議論も促します。

## AI業界の主要プレイヤー

### 主要企業
Google、OpenAI、Microsoft、IBMといった大手企業はAI開発を率いています。Googleは検索アルゴリズムやニューラルネットワークにおけるAI能力で革新を続け、OpenAIはGPTのような言語モデルで限界を押し広げています。MicrosoftとIBMはAI駆動のビジネスソリューションで大きな進歩を遂げており、進化する技術の風景への貢献を強調しています。

### 新興スタートアップ
大手企業を超えて、新興スタートアップは革新的なAIアプリケーションを導入し、伝統的な市場を混乱させ、重要な波を作り出しています。これらの機敏な企業はしばしばニッチな領域に焦点を当て、最先端の研究と技術を活用したテイラードAIソリューションを提供しています。

## 注目すべきAIニュース

### 画期的な技術と研究成果
AI分野は、機械学習アルゴリズムの進歩から様々な産業での革新的アプリケーションに至るまで、画期的な研究で満ちています。これらの発展は、以前は達成不可能と考えられていた新しい能力を解放するAIの可能性を強調しています。

### 法的最新情報
AIが日常生活にますます浸透する中で、法的措置は現在、AI倫理とデータ保護に焦点を合わせています。これらの規制は技術成長と社会的ニーズのバランスを取ることを目指し、倫理的考慮やプライバシー権を犠牲にすることなくAIの利益を保証します。

### 成功した実装
ビジネスの事例研究は、AIが操作効率と意思決定プロセスをどのように強化できるかを示す成功したAIの実装を強調しています。これらの例は、ワークフローにAIを統合しようとする企業にとって重要な学習ポイントとなります。

## ターゲットオーディエンス分析

### 興味と痛点
テクノロジー愛好家、ビジネス専門家、学生は皆、AIイノベーションと開発の未来に強い関心を示しています。しかし、AI倫理、データプライバシー、および潜在的な失職に関する広がる懸念が、AIアプリケーションの継続的な議論と理解の必要性を強調しています。

## 重要なポイント

### 生産性の向上
AIが産業全体の生産性を高める役割は否定できません。ルーチンタスクの自動化からデータ分析を通じた深い洞察の提供まで、この能力は前例のない効率と革新の舞台を整えています。

### AI倫理の議論
AIが進化を続ける中で、その使用に伴う倫理的考慮の重要性も増しています。これらの問題に焦点を当てた立法は、AI開発が社会的価値と倫理基準に沿うことを保証するために重要です。

### 未来の見通し
AIの未来の見通しは広大で強力であり、新しい可能性を解放し、部門間のやり取りを再定義することで社会を再構築することを約束します。

## 行動喚起
「Artificial Intelligence trends 2023」に関する情報を得るためにテクノロジーニュースレターを購読し続けましょう。フォーラムやソーシャルメディアを通じて「AI倫理と規制」に関する議論に参加し、法令の変更に先んじましょう。AI技術に詳しい教育ウェビナーやワークショップに参加することを検討し、このダイナミックな分野の最前線に居続けましょう。

AIの進歩に関するさらなる情報をご覧になりたい場合は、[Google AI](https://ai.google/)、[OpenAI](https://www.openai.com/)、[Microsoft AI](https://www.microsoft.com/en-us/ai)、および[IBM Watson](https://www.ibm.com/watson)を訪問するか、[MITのAI倫理研究](https://aiethics.mit.edu/)でAI倫理を探求してください。
```

このブログ記事は、AIの最新トレンド、業界の主要プレイヤー、および注目すべきニュースについての洞察に満ちた議論を通じて読者の興味を引くように設計されており、関連するSEOキーワードをシームレスに組み込んでいます。

In [42]:
result

CrewOutput(raw="The blog post has been refined for grammatical accuracy and alignment with the brand's voice as requested, presented in Markdown format. It provides a comprehensive overview of AI trends, news, industry players, and actionable insights.", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='1. 주어진 블로그 게시물(blog post)을 일본어로 번역합니다.\n', name=None, expected_output='번역된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공합니다.', summary='1. 주어진 블로그 게시물(blog post)을 일본어로 번역합니다.\n...', raw='Please provide the content of the blog post that needs translation, so I can accurately translate it into Japanese for you.\n```', pydantic=None, json_dict=None, agent='번역자 (Translator)', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='1. 인공지능에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.\n2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.\n3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "2025년 3월 2일 서울 날씨"
result = crew.kickoff(inputs={"topic": topic})

# Agent: 콘텐츠 작가 (Content Writer)
## Task: 1. 콘텐츠 계획(content plan)을 활용하여 2025년 3월 2일 서울 날씨에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.
2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.
3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.
4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.
5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).

# Agent: 콘텐츠 기획자 (Content Planner)
## Task: What are the SEO keywords we need to integrate naturally into the blog post about 2025년 3월 2일 서울 날씨, and can you provide more details on the brand's voice for consistency?


# Agent: 콘텐츠 기획자 (Content Planner)
## Final Answer: 
To effectively plan for the blog post about the weather in Seoul on March 2, 2025, incorporating relevant SEO keywords is crucial for maximizing search engine visibility. Here are some suggested keywords to integrate naturally into the content:

1. **2025 Seoul Weather**
2. **March 2 Weather Forecast Seoul**


In [17]:
from IPython.display import Markdown
Markdown(result.raw)

The comprehensive content plan for the 2025년 3월 2일 서울 날씨 blog post includes a detailed outline (introduction, key points, conclusion, call to action), an audience analysis outlining interests and pain points, a list of SEO keywords, and suggested resources for information and inspiration.